In [15]:
import ee
import os
import geemap
import json
from shapely.geometry import shape
import geopandas as gpd

In [16]:
ee.Authenticate()

True

In [17]:
ee.Initialize()

In [18]:
# region = 'AKCP_weekly'
region = 'YKD_weekly'
# region = 'YKD_weekly'
# region = 'YKF_weekly'
folder = 'projects/alpod-412314/assets/' + region

In [19]:
image_list = ee.data.listImages(folder)
image_list = image_list.get('images')

In [20]:
#print(image_list[0:5])
rando_img = ee.Image(image_list[0]['id'])
rando_img_stats = rando_img.reduceRegion(
    reducer = ee.Reducer.minMax(),
    scale = 10
)

rando_properties = rando_img.getInfo()['properties']
rando_coords = rando_properties['system:footprint']['coordinates']
print(rando_coords)
polygon = ee.Geometry.Polygon(rando_coords)

[[-165.84319482922518, 61.334407594596996], [-166.09958530214465, 61.33440757070055], [-166.4841708075412, 61.33440755767273], [-166.86894410533003, 61.33440750232537], [-166.86893836721399, 60.33619919540345], [-166.676463585461, 60.33615307524081], [-166.42007326744408, 60.33615308977307], [-166.22778045805273, 60.33619929212763], [-166.03548764518138, 60.33615308084961], [-165.7790971444423, 60.336153061499246], [-165.52270678371238, 60.33615305980325], [-165.3304139971633, 60.33619932740822], [-165.1381211612177, 60.33615306454182], [-164.94582836152594, 60.336199316844436], [-164.81745141921837, 60.33619922418118], [-164.8174455604192, 61.33440746584451], [-165.20221886181534, 61.33440753660442], [-165.5868044202101, 61.33440752947061], [-165.84319482922518, 61.334407594596996]]


In [21]:
target_year = '2018'
target_week = '27'

In [22]:
target_imgs = []
target_polygons = []

for image in image_list:
    img_id = image['id']
    year = img_id.split('_')[-2]
    week = img_id.split('_')[-1]

    if year == target_year:
        img_ee = ee.Image(img_id)
        img_properties = img_ee.getInfo()['properties']
        img_coords = img_properties['system:footprint']['coordinates']
        polygon_ee = ee.Geometry.Polygon(img_coords)
        
        target_imgs.append(img_ee)
        target_polygons.append(polygon_ee)

#print(len(target_imgs))

In [23]:
p1 = target_polygons[0]
p2 = target_polygons[1]

combined_footprint = p1.union(p2)

for i in range(2, len(target_polygons)):
    p = target_polygons[i]
    combined_footprint = combined_footprint.union(p)

In [24]:
# combined_footprint_info = combined_footprint.getInfo()
# geojson_footprint = json.dumps(combined_footprint_info)
# geometry = shape(json.loads(geojson_footprint))
# roi_gdf = gpd.GeoDataFrame([{'geometry': geometry, 'roi': f'{region}'}], crs="EPSG:4326")
# os.chdir('/Users/jmaze/Documents/projects/altimetry_lakes_v3/')
# roi_gdf.to_file(f'./data/ew_rois/{region}.shp')

In [25]:
target_collection = ee.ImageCollection(target_imgs)
target_mosaic = target_collection.mosaic()
clipped_mosaic = target_mosaic.clip(combined_footprint)

In [26]:
export_task = ee.batch.Export.image.toDrive(
    image=clipped_mosaic,
    description=f'week{target_week}_{region}',
    folder='ewebb_s2',
    fileNamePrefix='test_mosaic',
    region=combined_footprint,
    scale=10,
    fileFormat='GeoTIFF',
    maxPixels=1e13
)

#export_task.start()

In [27]:
viz1 = {
    'bands': ['water_occurance'],
    'min': 2,
    'max': 10,
    'palette': ['ff0000', '00ff00', '0000ff']
}

Map = geemap.Map(center=(40, -100), zoom=4)
Map.addLayer(target_mosaic, viz1, f'{region}, {target_week}')
Map.addLayer(combined_footprint, {}, 'roi_bounds')
Map


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [ ]:
binary_sum = weekly_sum.gt(3)

vectors = binary_sum.reduceToVectors(
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=10,
    maxPixels=1e13
)
# !!! Need to switch to a for-loop, becuase .getInfo() won't map to the server-side. 
def get_geojson(vectors):
    return ee.Feature(vectors).geometry().getInfo()

vector_geoms = vectors.map(get_geojson)

print(vector_geoms)

In [43]:
np_img = geemap.ee_to_numpy(clipped_mosaic)

60
